## Filtros FIR-LS con registro finito: Método de Covarianza

El objetivo del método **FIR-LS (Least Squares)** es ajustar un filtro FIR de orden $L-1$ que minimice el error entre una señal deseada $d(n)$ y una señal generada a partir de otra señal $x(n)$ filtrada por $h(n)$. La salida estimada del sistema es:

$$
\hat{d}(n) = \sum_{k=0}^{L-1} h(k) \, x(n - k)
$$

El error entre la señal deseada y la salida estimada es:

$$
e(n) = d(n) - \hat{d}(n) = d(n) - \sum_{k=0}^{L-1} h(k) \, x(n - k)
$$

El criterio de optimización es la minimización de la energía del error:

$$
\mathcal{E}(\mathbf{h}) = \sum_{n=i_0}^{i_f} |e(n)|^2
$$

Los límites $i_0$ e $i_f$ se eligen para que todas las muestras de $x(n-k)$ y $d(n)$ estén disponibles.

---

### Solución en términos matriciales

La solución óptima del filtro FIR se obtiene resolviendo el sistema de ecuaciones normales:

$$
\mathbf{R}_x \, \mathbf{h} = \mathbf{r}_{dx}
$$

donde:

- $\mathbf{R}_x \in \mathbb{C}^{L \times L}$ es la matriz de autocorrelación de la señal $x(n)$:

$$
r_x(k, l) = \sum_{n = i_0}^{i_f} x(n - l) \, x^*(n - k)
$$

- $\mathbf{r}_{dx} \in \mathbb{C}^L$ es el vector de correlación cruzada entre $d(n)$ y $x(n)$:

$$
r_{dx}(k) = \sum_{n = i_0}^{i_f} d(n) \, x^*(n - k)
$$

- $\mathbf{h} = [h(0), h(1), \dots, h(L-1)]^T$ es el vector de coeficientes del filtro.

---

### Solución alternativa usando matrices

Se puede construir la matriz $\mathbf{X}_{i_0}$ de regresores de la señal $x(n)$:

$$
\mathbf{X}_{i_0} =
\begin{bmatrix}
x(i_0) & x(i_0 - 1) & \cdots & x(i_0 - L + 1) \\
x(i_0 + 1) & x(i_0) & \cdots & x(i_0 - L + 2) \\
\vdots & \vdots & \ddots & \vdots \\
x(i_f) & x(i_f - 1) & \cdots & x(i_f - L + 1)
\end{bmatrix}
$$

Y el vector:

$$
\mathbf{d}_{i_0} =
\begin{bmatrix}
d(i_0) \\
d(i_0 + 1) \\
\vdots \\
d(i_f)
\end{bmatrix}
$$

Entonces:

- $\mathbf{R}_x = \mathbf{X}_{i_0}^H \mathbf{X}_{i_0}$
- $\mathbf{r}_{dx} = \mathbf{X}_{i_0}^H \mathbf{d}_{i_0}$

---

### Error mínimo alcanzado

El valor mínimo de la energía del error se puede expresar como:

$$
\mathcal{E}_{\min} = \|\mathbf{d}_{i_0}\|^2 - \sum_{k=0}^{L-1} h(k) \, r_{dx}^*(k)
$$

o equivalentemente:

$$
\mathcal{E}_{\min} = r_d(0, 0) - \sum_{k=0}^{L-1} h(k) \, r_{dx}^*(k)
$$

donde:

$$
r_d(0, 0) = \sum_{n = i_0}^{i_f} |d(n)|^2 = \|\mathbf{d}_{i_0}\|^2
$$

---

Este método no impone ninguna suposición sobre los datos fuera del intervalo de observación, a diferencia del método de autocorrelación. Es por tanto más adecuado en condiciones de ventana finita, especialmente en aplicaciones prácticas como el diseño de filtros adaptativos o estimación de sistemas.


In [ ]:
%reset

import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io.wavfile import write
from scipy.linalg import toeplitz  # comando de generacion matrices Toeplitz
from scipy.linalg import inv  # comando de inversion de matrices


def coeficientes_AR_COV(x, p):
    """
    Estima los coeficientes AR(p) mediante el método de covarianza.

    Parámetros:
    -----------
    x : array_like
        Señal de entrada, x(n) para n = 0, 1, ..., N-1.
    p : int
        Orden del modelo AR (número de polos).

    Devuelve:
    --------
    a : ndarray
        Coeficientes del modelo AR en la forma: a[0] = 1, a[1], ..., a[p],
        de manera que:
            x(n) + ∑[k=1..p] a[k] x(n-k) = e(n)
    E_p : float
        Error cuadrático mínimo (energía del residuo) obtenido en la aproximación
        por mínimos cuadrados, análogo a la ecuación (4.126) o (4.132) en el libro.
    """
    
    # ?------------------------------------------------------
    # ? 1. Definiciones básicas
    # ?------------------------------------------------------
    N = len(x)  # * Longitud total de la señal
    if p >= N:
        raise ValueError("El orden p debe ser menor que la longitud de la señal.")

    # ?------------------------------------------------------
    # ? 2. Construcción de la matriz de regresores y del vector objetivo
    # ?    Para n = p, p+1, ..., N-1, se predice x(n) en función de:
    # ?    [x(n-1), x(n-2), ..., x(n-p)].
    # ?------------------------------------------------------
    
    # * Construir la primera columna de X:
    #   Se toman los valores desde x(p-1) hasta x(N-2) (inclusive).
    columna = x[p - 1 : N-1]      #  x(p-1), x(p+1), ..., x(N-2)  La longitud es N - p
    fila = x[:p][::-1]          # x(p-1), x(p-2), ..., x(0)     La longitud es p
    
    # * Construir la matriz de regresores X (dimensión (N-p) x p)
    X = toeplitz(columna, fila)
    
    # * Definir el vector de salida (target) d:
    d = x[p : N]    # d = [x(p), x(p+1), ..., x(N-1)]
    
    # ?------------------------------------------------------
    # ? 3. Resolución de las ecuaciones normales de covarianza
    # ?    Se resuelve: (X^H X) a_vec = - X^H d, donde a_vec = [a(1), ..., a(p)]
    # ?------------------------------------------------------
    X_H = X.conj().T                # * Transpuesta conjugada de X (útil para datos complejos)
    R_x = X_H @ X                   # * Matriz de autocorrelación de los regresores
    r_x = X_H @ d                   # * Vector de correlación cruzada con la salida
    A_partial = -inv(R_x) @ r_x     # ! Solución para los coeficientes a(1) ... a(p)
    
    # ?------------------------------------------------------
    # ? 4. Cálculo del error cuadrático mínimo E_p
    # ?    E_p = ∑[n=p]^(N-1) |e(n)|^2, con e(n) = x(n) + ∑[k=1]^(p) a[k] x(n-k)
    # ?------------------------------------------------------
    E_p = np.dot(d, d) + np.dot(A_partial, r_x)
    
    # ?------------------------------------------------------
    # ? 5. Construcción del vector de coeficientes del modelo AR
    # ?    Se define el polinomio:
    # ?      A(z) = 1 + ∑[k=1..p] a(k) z^{-k}
    # ?    Por convención, se añade a[0] = 1.
    # ?------------------------------------------------------
    a = np.concatenate(([1.0], A_partial))
    
    return a, E_p
